## Introduction

This project will be usefull to people who want to migrate from one city (Moscow, Russia) to another (New York, USA). It will allow to understand similarity of two selected boroughs - one in Moscow and one in New York. Similarity will be based on venue clusterization. 

## Data
We will be using Foursquare to get venue data. Foursquare will be used to get data for all neighborhoods in selected borough. We will get some data about boroughs and neighborhoods from the internet and some data will be taken from previous Coursera projects.

Loading libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
# url = 'https://en.wikipedia.org/wiki/Administrative_divisions_of_Moscow'
# response = requests.get(url)
# soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
# not used
# Getting Okrug names
# divs = soup.find_all("th", {"class": "navbox-group"})
# for el in divs:
#     string = el.text.strip()
#     if string[-5:] == 'Okrug':
#         print(string)

In [4]:
# okrug_names = []
# okrug_urls = []
# divs = soup.find_all("th", {"class": "navbox-group"})
# for el in divs[:12]:
# #     okrug_urls.append(''.join(['https://en.wikipedia.org', re.findall(r'a href="(.*)" title', str(el))]))
#     okrug_urls.append(''.join(['https://en.wikipedia.org', re.search(r'a href="(.*)" ', str(el)).group(1)]))
#     if el.text.endswith('Okrug'):
#         okrug_names.append(el.text)
# df = pd.DataFrame(data=None, index=None, columns=['Okrug_names', 'Okrug_urls'], dtype=None, copy=False)
# df['Okrug_names'] = okrug_names
# df['Okrug_urls'] = okrug_urls
# df

In [5]:
# lat_arr = []
# lng_arr = []
# for el in df.Okrug_urls:
#     response = requests.get(el)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     lat, lng = soup.find("span", {"class": "geo"}).text.split('; ') #Get Okrug coordinates
#     lat_arr.append(lat)
#     lng_arr.append(lng)
# df['Latitude'] = lat_arr
# df['Longitude'] = lng_arr
# df

In [6]:
# import folium # map rendering library
# from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# address = 'Moscow, RU'

# geolocator = Nominatim(user_agent="ny_explorer")
# location = geolocator.geocode(address)
# latitude = location.latitude
# longitude = location.longitude
# print('The geograpical coordinates of Moscow are {}, {}.'.format(latitude, longitude))

In [7]:
# map_Moscow = folium.Map(location=[latitude, longitude], zoom_start=9)
# for Okrug_name, lat, lng in zip(df['Okrug_names'], \
#                              df['Latitude'], \
#                              df['Longitude']):
#     label = '{}'.format(Okrug_name)
#     label = folium.Popup(label, parse_html=True)
#     folium.CircleMarker(
#         [lat, lng],
#         radius=5,
#         popup=label,
#         color='green',
#         fill=True,
#         fill_color='#863100',
#         fill_opacity=0.7,
#         parse_html=False).add_to(map_Moscow)  
# map_Moscow

In [8]:
# def shortener(s):
#     return ('').join(re.findall(r'\b\w', s))

# df['Short_Okrugs'] = df.Okrug_names.apply(shortener)
# df = df[['Okrug_names', 'Short_Okrugs', 'Latitude', 'Longitude', 'Okrug_urls']]
# df

In [9]:
# for url in df.Okrug_urls:
# # url = df['Okrug_urls'][1]
#     response = requests.get(url)
#     # soup = BeautifulSoup(response.text, 'html.parser')
#     soup = BeautifulSoup(str(re.findall(r'</sup>..?</p>(.*)References', str(response.content))), 'html.parser')
#     links = soup.select('ul li a')
#     names = []
#     urls = []
#     for el in links:
#         urls.append(''.join(['https://en.wikipedia.org', el.get('href')]))
#         names.append(el.text)
#     print(names)

In [10]:
# url = 'https://en.wikipedia.org/wiki/Administrative_divisions_of_Moscow'
# response = requests.get(url)
# soup = BeautifulSoup(response.text, 'html.parser')
# for table in soup.find_all('table', {'class' : 'wikitable'})[1:]:
#     table_rows = table.find_all('tr')
#     for tr in table_rows:
#         td = tr.find_all('td')
#         row = [i.text for i in td]
#         print(row)

In [11]:
# url = 'https://en.wikipedia.org/wiki/Administrative_divisions_of_Moscow'
# b=0
# response = requests.get(url)
# soup = BeautifulSoup(response.text, 'html.parser')
# for table in soup.find_all('table', {'class' : 'wikitable'})[1:]:
#     table_rows = table.find_all('tr')
#     for tr in table_rows:
#         td = tr.find_all('td')
#         for el in td:
#             a = el.find_all('a', {'class' : ''})
#             if a:
#                 print(a[0].text, a[0].get('href'))
#                 b +=1
# print(b)

In [12]:
# url = 'https://en.wikipedia.org/wiki/Administrative_divisions_of_Moscow'
# response = requests.get(url)
# soup = BeautifulSoup(response.text, 'html.parser')
# # table = soup.select('table', {'class' : 'nowraplinks mw-collapsible mw-collapsed navbox-inner'})
# table = soup.find('table', {'class' : 'nowraplinks mw-collapsible mw-collapsed navbox-inner'})
# for el in table.find_all('a')[4:]:
#     print(el.text)

In [13]:
# import requests
# import json
# overpass_url = "http://overpass-api.de/api/interpreter"
# overpass_query = """
# [out:json];
# area["int_ref"="RU-MOW"];
# out;
# """
# response = requests.get(overpass_url, 
#                         params={'data': overpass_query})
# data = response.json()
# data

In [5]:
import requests
import json
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["addr:country"="RU"]["addr:region"="Москва"][admin_level=5];
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()
Okrug_Names = []
wikidata = []
for i in range(len(data['elements'])):
    Okrug_Names.append(data['elements'][i]['tags']['name:en'])
    wikidata.append(data['elements'][i]['tags']['wikidata'])
dff = pd.DataFrame(data=None, index=None, columns=['Okrug', 'Wikidata'], dtype=None, copy=False)
dff['Okrug'] = Okrug_Names
dff['Wikidata'] = wikidata
dff

,Okrug,Wikidata
0,Northern Administrative Okrug,Q462016
1,Western Administrative Okrug,Q462475
2,North-Western Administrative Okrug,Q462490
3,North-Eastern Administrative Okrug,Q462247
4,South-Eastern Administrative Okrug,Q121979
5,Southern Administrative Okrug,Q462377
6,South-Western Administrative Okrug,Q462400
7,Eastern Administrative Okrug,Q307234
8,Zelenogradsky Administrative Okrug,Q207695
9,Central Administrative Okrug,Q190412


In [18]:
baseurl = 'https://www.wikidata.org/wiki/'
okrug_lat = []
okrug_lon = []
for el in dff['Wikidata']:
    url = ''.join([baseurl, el])
    response = requests.get(url).text
    okrug_lat.append(re.findall(r'data-lat="(\d+\.\d+)" ', response)[0])
    okrug_lon.append(re.findall(r'data-lon="(\d+\.\d+)" ', response)[0])
dff['okrug_lat'] = okrug_lat
dff['okrug_lon'] = okrug_lon
dff

,Okrug,Wikidata,okrug_lat,okrug_lon
0,Northern Administrative Okrug,Q462016,55.83333333,37.51666667
1,Western Administrative Okrug,Q462475,55.7247,37.4856
2,North-Western Administrative Okrug,Q462490,55.81666667,37.43333333
3,North-Eastern Administrative Okrug,Q462247,55.83333333,37.61666667
4,South-Eastern Administrative Okrug,Q121979,55.666666666667,37.616666666667
5,Southern Administrative Okrug,Q462377,55.63333333,37.66666667
6,South-Western Administrative Okrug,Q462400,55.65,37.53333333
7,Eastern Administrative Okrug,Q307234,55.78333333,37.76666667
8,Zelenogradsky Administrative Okrug,Q207695,55.997777777778,37.190277777778
9,Central Administrative Okrug,Q190412,55.75,37.616666666667


In [46]:
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Moscow, RU'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Moscow are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Moscow are 55.7504461, 37.6174943.


In [40]:
aa = []
ss = []
for el in dff['Okrug']:
    address = '%s, RU' % el

    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of Moscow are {}, {}.'.format(latitude, longitude))
    aa.append(latitude)
    ss.append(longitude)
dff['aa'] = aa
dff['ss'] = ss

The geograpical coordinates of Moscow are 55.83365735, 37.5258086336107.
The geograpical coordinates of Moscow are 55.700349, 37.4675396700155.
The geograpical coordinates of Moscow are 55.82674165, 37.4354953687849.
The geograpical coordinates of Moscow are 55.870168, 37.6436056370616.
The geograpical coordinates of Moscow are 55.6993352, 37.7652478530234.
The geograpical coordinates of Moscow are 55.64796495, 37.6440826067665.
The geograpical coordinates of Moscow are 55.60284985, 37.5409266684086.
The geograpical coordinates of Moscow are 55.78690375, 37.7527271060133.
The geograpical coordinates of Moscow are 55.98496825, 37.2056124309019.
The geograpical coordinates of Moscow are 55.7538479, 37.5954635714559.
The geograpical coordinates of Moscow are 55.5501575, 37.3873794326508.
The geograpical coordinates of Moscow are 55.35347845, 37.1788677203488.


In [49]:
map_Moscow = folium.Map(location=[latitude, longitude], zoom_start=9)
for Okrug_name, lat, lng in zip(dff['Okrug'], \
                             dff['aa'], \
                             dff['ss']):
    label = '{}'.format(Okrug_name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#863100',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Moscow)  
map_Moscow

In [50]:
# overpass_url = "http://overpass-api.de/api/interpreter"
# overpass_query = """
# [out:json];
# relation['name:en' = 'Moscow'][type="boundary"];
# out;
# """
# response = requests.get(overpass_url, 
#                         params={'data': overpass_query})
# data = response.json()
# data


# overpass_url = "http://overpass-api.de/api/interpreter"
# overpass_query = """
# [out:json];
# area["addr:country"="RU"]["addr:region"="Москва"][admin_level=5];
# rel(area)[admin_level=8][boundary=administrative];
# out;
# """
# response = requests.get(overpass_url, 
#                         params={'data': overpass_query})
# data = response.json()
# data
di = []
dd = {}
for el in Okrug_Names:
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = """
    [out:json];
    area['name:en' = '%s'][type="boundary"];
    rel(area)[admin_level=8][boundary=administrative];
    out;
    """ % el
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    data = response.json()
    print(el)
    di = []
    for k in range(len(data['elements'])):
        
        try:
            di.append(data['elements'][k]['tags']['name:en'])
        except:
            di.append(data['elements'][k]['tags']['name'])
    dd[el] = di
dd

Northern Administrative Okrug
Western Administrative Okrug


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [51]:
full = []
for el in dd.keys():
    for el2 in dd[el]:
        full.append([el, el2])
dff = pd.DataFrame(data=full, index=None, columns=['Okrug', 'District'], dtype=None, copy=False)
dff

,Okrug,District
0,Northern Administrative Okrug,Levoberezhny District
1,Northern Administrative Okrug,Khovrino District
2,Northern Administrative Okrug,Zapadnoye Degunino District
3,Northern Administrative Okrug,Timiryazevsky District
4,Northern Administrative Okrug,Koptevo District
5,Northern Administrative Okrug,Dmitrovsky District
6,Northern Administrative Okrug,Sokol District
7,Northern Administrative Okrug,Savyolovsky District
8,Northern Administrative Okrug,Vostochnoye Degunino District
9,Northern Administrative Okrug,Golovinsky District
